In [14]:
from transformers import pipeline  # For using pre-trained models
import numpy as np  # For numerical operations on arrays
from PIL import Image  # For image manipulation
from LookBuilderPipeline.resize.resize import resize_images
import glob



# Initialize the segmentation model using a pre-trained model from Hugging Face
segmenter = pipeline(model="mattmdjaga/segformer_b2_clothes")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [30]:
# # Load the image from the specified path
# seg_img = Image.open(image_path)

# Use the segmenter to get segments from the image


image_folder=glob.glob("/Users/apple/WRK/modegen/MPI-cog/ComfyUI/input/jcrew_orig/*") 

for j,i in enumerate(image_folder):
    seg_img = Image.open(i)
    
    segments = segmenter(seg_img)
    

    # Define the labels for the segments we want to include
    segment_include = ["Upper-clothes", "Skirt", "Pants", "Dress", "Belt", "Bag", "Scarf", "Left-shoe", "Right-shoe","Bag"]

    # # Extend the segments to include additional options if specified
    # if additional_option in ["shoe"]:
    #     segment_include.extend(["Left-shoe", "Right-shoe"])
    # if additional_option in ["bag"]:
    #     segment_include.extend(["Bag"])

    # if inverse==False:
    #     # Create a list of masks for the included segments
    #     mask_list = [np.array(s['mask']) for s in segments if s['label'] in segment_include]
    # else:
    mask_list = [np.array(s['mask']) for s in segments if s['label'] in segment_include]


    # Initialize the final mask with the first mask in the list
    final_mask = np.array(mask_list[0])

    # Combine all masks into a single final mask
    for mask in mask_list:
        current_mask = np.array(mask)
        final_mask = final_mask + current_mask  # Add the current mask to the final mask

    # Create a copy of the final mask for later use
    final_array = final_mask.copy()

    # Convert the final mask to a PIL Image
    final_mask = Image.fromarray(final_mask)
    final_mask.save('inv/final_mask_image'+str(j)+'.png')

    
    # Add the mask as an alpha channel to the original image
    seg_img.putalpha(final_mask)
    seg_img.save('inv/segmented_image'+str(j)+'.png')
    
    im2 = final_mask.copy()
    seg_img.putalpha(final_mask)
    final_mask.paste(seg_img, final_mask)
    final_mask.save('inv/black_image'+str(j)+'.png')


In [ ]:

.resize(pose.size)